## Download Dataset from following link

In [0]:
##https://www.floydhub.com/emilwallner/datasets/colornet

drive  sample_data


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd '/content/drive/My Drive/app/colorize&restore-images/BetaVersion/'

/content/drive/My Drive/app/colorize&restore-images/BetaVersion


In [0]:
!ls -alh

total 682M
-rw------- 1 root root 1.8K Jan  2 09:09 BetaVersion.ipynb
-rw------- 1 root root 682M Jan  2 08:41 emilwallner-datasets-colornet-2.tar


In [0]:
!tar -xvf emilwallner-datasets-colornet-2.tar

./
./inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5
./testdata/
./testdata/Validate/
./testdata/Validate/0yTHvf.jpg
./testdata/Validate/3YFtxe.jpg
./testdata/Validate/9KfZez.jpg
./testdata/Validate/6v14hm.jpg
./testdata/Validate/1QejlL 4.11.06 PM.jpg
./testdata/Validate/0fAtAB.jpg
./testdata/Validate/4r3yxj.jpg
./testdata/Validate/7Vizcm.jpg
./testdata/Train/
./testdata/Train/9KkoOf.jpg
./testdata/Train/8ntGqA.jpg
./testdata/Train/8oZO2i.jpg
./testdata/Train/30gdlQ.jpg
./testdata/Train/29Pkqp.jpg
./testdata/Train/27gIpN.jpg
./testdata/Train/1PFDZe.jpg
./testdata/Train/0qADtP.jpg
./testdata/Train/8X0jDX.jpg
./testdata/Train/11Se02.jpg
./images/
./images/Test/
./images/Test/xpQYWH.jpg
./images/Test/DP2xMv.jpg
./images/Test/EJiG7k.jpg
./images/Test/pshaly.jpg
./images/Test/photo-1476836349418-180f91b52141.jpeg
./images/Test/PEhlWA.jpg
./images/Test/photo-1499443186375-356d65c84cd5.jpeg
./images/Test/FnrvWN.jpg
./images/Test/P7nBAi.jpg
./images/Test/d4nyFl.jpg
./images/Test/phot

In [0]:
!ls -alh

total 897M
-rw------- 1 root root 319K Jan  2 09:24 BetaVersion.ipynb
-rwx------ 1 root root 6.1K Jan  1  1970 .DS_Store
-rw------- 1 root root 682M Jan  2 08:41 emilwallner-datasets-colornet-2.tar
drwx------ 4 root root 4.0K Jan  2 09:18 images
-rwx------ 1 root root 215M Jan  1  1970 inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5
drwx------ 4 root root 4.0K Jan  1  1970 testdata


In [0]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [0]:
# Get images
X = []
for filename in os.listdir('images/Train/'):
    X.append(img_to_array(load_img('images/Train/'+filename)))
X = np.array(X, dtype=float)

In [0]:
# train and test data
split = int(0.95 * len(X))
Xtrain = X[:split]
Xtrain = 1.0/255 * Xtrain

In [0]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [0]:
# image generator
datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, rotation_range=20, horizontal_flip=True)

In [0]:
# Generate training data
batch_size = 10

def image_a_b_gen(batch_size):
  for batch in datagen.flow(Xtrain, batch_size=batch_size):
    lab_batch = rgb2lab(batch)
    X_batch = lab_batch[:,:,:,0]
    Y_batch = lab_batch[:,:,:,1:] / 128
    yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [0]:
tensorboard = TensorBoard(log_dir="output/first_run")

In [0]:
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=100, steps_per_epoch=10)

In [0]:
# save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [0]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

In [0]:
color_me = []
for filename in os.listdir('images/Test/'):
    color_me.append(img_to_array(load_img('images/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

